In [ ]:
#|default_exp tools

In [ ]:
#|export
from pathlib import Path
from shlex import split
from subprocess import run

In [ ]:
#|hide
from fastcore.test import test_eq

# Computer Tools

> Helpful tools for running cli commands and reading, modifying, and creating files in python. This is used primarily for AI's in tool loops for automating tasks involving the filesystem.

## Bash Tools

A utility function for running any cli command from python and return the output as a string.

In [ ]:
#| export
def run_cmd(cmd:str, argstr:str):
    "Run the cli `cmd` with the given arguments. Returns the stdout and stderr as a concatted string."
    res = run([cmd] + split(argstr), capture_output=True, text=True)
    return res.stdout + '\n' + res.stderr

With this little function, we can now run any cli command:

In [ ]:
print(run_cmd('ls', '')[:128])

_parallel_win.ipynb
_quarto.yml
00_test.ipynb
000_tour.ipynb
01_basics.ipynb
02_foundation.ipynb
03_xtras.ipynb
03a_parallel.ipy


Let's create some useful functions from this that will allow for searching, reading and modifing content on the file system.

In [ ]:
#| export
def rg(argstr:str):
    "Run the `rg` command with the args in `argstr` (no need to backslash escape)"
    return run_cmd('rg', argstr)

In [ ]:
print(rg('"fastcore" CNAME'))

fastcore.fast.ai




In [ ]:
#| export
def sed(argstr:str):
    "Run the `sed` command with the args in `argstr` (e.g for reading a section of a file)"
    return run_cmd('sed', argstr)

In [ ]:
print(sed('-n "1,5 p" _quarto.yml'))

project:
  type: website
  pre-render: 
    - pysym2md --output_file apilist.txt fastcore
  post-render: 




## Text Editor Tools

Python implementations of the text editor tools from [Anthropic](https://docs.claude.com/en/docs/agents-and-tools/tool-use/text-editor-tool). These tools are especially useful in an AI's tool loop. See [`claudette`](https://claudette.answer.ai/text_editor.html) for examples.

In [ ]:
#| export
def view(path:str, view_range:tuple[int,int]=None, nums:bool=False):
    'View directory or file contents with optional line range and numbers'
    try:
        p = Path(path).expanduser().resolve()
        if not p.exists(): return f'Error: File not found: {p}'
        if p.is_dir():
            res = [str(f) for f in p.glob('**/*') 
                   if not any(part.startswith('.') for part in f.relative_to(p).parts)]
            return f'Directory contents of {p}:\n' + '\n'.join(res)
        
        lines = p.read_text().splitlines()
        s,e = 1,len(lines)
        if view_range:
            s,e = view_range
            if not (1 <= s <= len(lines)): return f'Error: Invalid start line {s}'
            if e != -1 and not (s <= e <= len(lines)): return f'Error: Invalid end line {e}'
            lines = lines[s-1:None if e==-1 else e]
            
        return '\n'.join([f'{i+s-1:6d} │ {l}' for i,l in enumerate(lines,1)] if nums else lines)
    except Exception as e: return f'Error viewing file: {str(e)}'

With `view` you can show the contents of files and directories. What's particularly nice is that you can also specify line rangs and whether to have the output contain line numbers:

In [ ]:
print(view('_quarto.yml', (1,5), nums=True))

     1 │ project:
     2 │   type: website
     3 │   pre-render: 
     4 │     - pysym2md --output_file apilist.txt fastcore
     5 │   post-render: 


Here's what the output looks like when viewing a directory:

In [ ]:
print(view('.')[:128])

Directory contents of /Users/nathan/aai-ws/fastcore/nbs:
/Users/nathan/aai-ws/fastcore/nbs/llms.txt
/Users/nathan/aai-ws/fastcor


In [ ]:
#| export
def create(path: str, file_text: str, overwrite:bool=False) -> str:
    'Creates a new file with the given content at the specified path'
    try:
        p = Path(path)
        if p.exists():
            if not overwrite: return f'Error: File already exists: {p}'
        p.parent.mkdir(parents=True, exist_ok=True)
        p.write_text(file_text)
        return f'Created file {p} containing:\n{file_text}'
    except Exception as e: return f'Error creating file: {str(e)}'

Now here is a function for creating files with specific contents:

In [ ]:
print(create('test.txt', 'Hello, world!'))

Created file test.txt containing:
Hello, world!
     1 │ Hello, world!


NB: These tools have special behavior around errors. Since these have been speficially designed for work with LLMs, any exceptions created from there use is returned as a string to help them debug their work.

In [ ]:
print(create('test.txt', 'Hello, world!'))

Error: File already exists: test.txt


In [ ]:
f = Path('test.txt')
test_eq(f.exists(), True)
print(view(f, nums=True))

     1 │ Hello, world!


Of course, you don't want to only be able to create new files, therefore, we have these next two functions for modifying their content:

In [ ]:
#| export
def insert(path: str, insert_line: int, new_str: str) -> str:
    'Insert new_str at specified line number'
    try:
        p = Path(path)
        if not p.exists(): return f'Error: File not found: {p}'
            
        content = p.read_text().splitlines()
        if not (0 <= insert_line <= len(content)): return f'Error: Invalid line number {insert_line}'
            
        content.insert(insert_line, new_str)
        new_content = '\n'.join(content)
        p.write_text(new_content)
        return f'Inserted text at line {insert_line} in {p}.\nNew contents:\n{new_content}'
    except Exception as e: return f'Error inserting text: {str(e)}'

In [ ]:
insert(f, 0, 'Let\'s add a new line')
print(view(f, nums=True))

     1 │ Let's add a new line
     2 │ Hello, world!


In [ ]:
#| export
def str_replace(path: str, old_str: str, new_str: str) -> str:
    'Replace first occurrence of old_str with new_str in file'
    try:
        p = Path(path)
        if not p.exists(): return f'Error: File not found: {p}'
            
        content = p.read_text()
        count = content.count(old_str)
        
        if count == 0: return 'Error: Text not found in file'
        if count > 1: return f'Error: Multiple matches found ({count})'
            
        new_content = content.replace(old_str, new_str, 1)
        p.write_text(new_content)
        return f'Replaced text in {p}.\nNew contents:\n{new_content}'
    except Exception as e: return f'Error replacing text: {str(e)}'

In [ ]:
str_replace(f, 'new line', '')
print(view(f, nums=True))
f.unlink()

     1 │ Let's add a 
     2 │ Hello, world!


# Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()